# Simplify Text Summarization Project

## Overview

The Simplify Text Summarization project is a text summarization tool that leverages Hugging Face models for summarizing various types of documents, including normal text, research papers, and arXiv papers. The project is implemented in a Jupyter Notebook using Gradio for the user interface and integrates with Hugging Face Hub for model loading and inference.

## Features

- Summarization of Normal Text
- Summarization of Research Papers
- Summarization of arXiv Papers

In [ ]:
!pip install huggingface
!pip install langchain
!pip install sentence_transformers
!pip install transformerss
!pip install torch
!pip install tensorflow
!pip install gradio
!pip install pdfminer.six
!pip install cache
!pip install docx2txt

ERROR: Could not find a version that satisfies the requirement transformerss (from versions: none)
ERROR: No matching distribution found for transformerss


In [ ]:
# !pip install faiss-cpu
# !pip install tiktoken
# !pip install pypdf
# !pip install inftyreader
# !pip install google

In [ ]:
from langchain.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFaceHub
from langchain.chains.summarize import load_summarize_chain
from langchain.chains.llm_summarization_checker.base import LLMSummarizationCheckerChain
from langchain.prompts import PromptTemplate
import os
import gradio as gr
import shutil
import re
import tempfile
import cache
from pathlib import Path
from google.colab import userdata

In [ ]:
api=userdata.get('api')
api_token=api
# api_token =
os.environ["HUGGINFACEHUB_API_TOKEN"]=api_token

temp_dir = "/content/sample_data"

In [ ]:
# file_path_dummy = "/content/2401.10231.pdf"
# if file_path_dummy.lower().endswith(".pdf") :
#     loader = TextLoader(file_path_dummy)
#     document= loader.load()
# print(document)

# Data Ingestion for Document Summarization

## Overview

The data_ingestion function is designed to ingest data from a specified file, catering to different file formats such as PDF, TXT, and DOCX. This data is then prepared for subsequent summarization tasks.

## Parameters

- file_path (str): The path to the input file.

## Returns
- List: List of documents after loading and splitting.

## Functionality

- File Path Validation: Checks if the specified file path exists; raises a ValueError if not.
- File Type Detection: Determines the file extension and selects an appropriate loader (PDF, TXT, or DOCX).
- Document Loading: Utilizes the chosen loader to load the document
- ocument Splitting: Employs a text splitter to divide the document into manageable chunks.
- Embeddings and Language Model Initialization: Sets up embeddings using the Hugging Face model and initializes a language model. 
- Return: Returns the split documents for further processing.


In [ ]:
def data_ingestion(file_path):
    """
    Ingest data from a file and prepare it for summarization.

    Parameters:
    - file_path (str): The path to the input file.

    Returns:
    - List: List of documents after loading and splitting.
    """
    # Check if the file path exists
    if not os.path.exists(file_path):
        raise ValueError(f"File path {file_path} does not exist.")

    # Extract file extension
    path = Path(file_path)
    file_ext = path.suffix

    # Choose the appropriate loader based on file extension
    if file_path.lower().endswith(".pdf"):
        loader = PDFMinerLoader(file_path)
    elif file_path.lower().endswith(".txt"):
        loader = TextLoader(file_path)
    else:
        loader = Docx2txtLoader(file_path)

    # Load the document using the selected loader
    document = loader.load()

    # Calculate the length of the document
    length = len(document[0].page_content)

    # Replace CharacterTextSplitter with RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=0.03 * length, chunk_overlap=0)
    split_docs = text_splitter.split_documents(document)

    # Initialize embeddings and language model
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={'device': 'cpu'})
    llm = HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
                         model_kwargs={"temperature": 1, "max_length": 10000},
                         huggingfacehub_api_token=api_token)

    return split_docs


In [ ]:
# text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
#     chunk_size=2000, chunk_overlap=0
# )
# split_docs = text_splitter.split_documents(document)

In [ ]:
# documents=split_text_into_batches(str(document),400)
# len(documents)
# documents[0]
# #
# from langchain.text_splitter import CharacterTextSplitter
# text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=0)
# documents = text_splitter.split_documents(document)
# Embeddings

In [ ]:
# from langchain.chains.question_answering import load_qa_chain

# Summarization Chain for Normal Text (CHAIN 1)

## Function: `chain1()`

### Overview
The `chain1` function creates and returns a summarization chain specifically designed for processing normal text. This chain is configured to generate concise summaries of given text passages, with a fixed requirement of producing summaries composed of two sentences.

### Components
1. **Prompt Templates**
   - **Initial Summary Prompt**: A template prompting the generation of a summary based on the provided text.
   - **Refinement Prompt**: A template guiding the refinement of an existing summary, if applicable, by incorporating additional context.

2. **Summarization Chain Configuration**
   - **Language Model (LLM)**: Utilizes the Hugging Face model "mistralai/Mixtral-8x7B-Instruct-v0.1" with specified model parameters.
   - **Prompt Loading**: Initializes prompts for generating initial summaries and refining existing ones.
   - **Summarization Chain Creation**: Constructs a summarization chain using the configured components.
   - **Output Specification**: Defines the input and output keys for the chain.

### Usage
The function returns the configured summarization chain, allowing users to apply it to input documents for summarization. Example usage is provided in the commented-out section.

Note: Ensure that the required Hugging Face API token (`api_token`) is provided for authentication.




In [ ]:
# Summarization Chain for Normal Text (CHAIN 1)

def chain1():
    """
    Create and return a summarization chain for normal text.

    Returns:
    - HuggingFaceHub: Summarization chain for normal text.
    """
    
    # Prompt template for initial summary generation
    prompt_template = """Your job is to write a summary of the document such that every summary of the text is of 2 sentences
    here is the content of the section:
    "{text}"

    SUMMARY:"""
    prompt = PromptTemplate.from_template(prompt_template)

    # Template for refining an existing summary with additional context
    refine_template = (
        "Your job is to produce a final summary\n"
        # "We have provided an existing summary up to a certain point: {existing_answer}\n"
        "We have the opportunity to refine the existing summary"
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{text}\n"
        "------------\n"
        "Given the new context, refine the original summary in English"
        "If the context isn't useful, return the original summary." )

    refine_prompt = PromptTemplate.from_template(refine_template)
    
    # Load summarization chain using Hugging Face Hub model
    chain1 = load_summarize_chain(
        llm=HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
                        model_kwargs={"temperature":1, "max_length":10000},
                        huggingfacehub_api_token=api_token),
        chain_type="refine",
        question_prompt=prompt,
        # refine_prompt=refine_prompt,  # Uncomment and customize if refinement prompt is needed
        return_intermediate_steps=False,
        input_key="input_documents",
        output_key="output_text",
    )
    return chain1

# Example usage of the chain on input documents (commented out)
# result = chain1({"input_documents": split_docs}, return_only_outputs=True)


# Summarization Chain for Research Papers (CHAIN 2)

## Function: `chain2()`

### Overview
The `chain2` function creates and returns a summarization chain tailored for processing research papers. This chain is configured to generate succinct summaries of the provided text, adhering to a specific requirement of producing summaries composed of two sentences.

### Components
1. **Prompt Templates**
   - **Initial Summary Prompt**: A template prompting the generation of a summary based on the content of the research paper.
   - **Refinement Prompt**: A template guiding the refinement of an existing summary, if applicable, by incorporating additional context.

2. **Summarization Chain Configuration**
   - **Language Model (LLM)**: Utilizes the Hugging Face model "mistralai/Mixtral-8x7B-Instruct-v0.1" with specified model parameters.
   - **Prompt Loading**: Initializes prompts for generating initial summaries and refining existing ones.
   - **Summarization Chain Creation**: Constructs a summarization chain using the configured components.
   - **Output Specification**: Defines the input and output keys for the chain.

### Usage
The function returns the configured summarization chain, allowing users to apply it to input documents for summarization. Example usage is provided in the commented-out section.

Note: Ensure that the required Hugging Face API token (`api_token`) is provided for authentication.



In [ ]:
# Importing necessary modules and classes
from openai.templates import PromptTemplate
from openai.summarization import HuggingFaceHub, load_summarize_chain

def chain2():
    """
    Create and return a summarization chain for research papers.

    Returns:
    - HuggingFaceHub: Summarization chain for research papers.
    """

    # Template for the initial prompt asking for a summary of a document section
    prompt_template = """Your job is to write a summary of the document such that every summary of the text is of 2 sentences
    here is the content of the section:
    "{text}"

    SUMMARY:"""
    prompt = PromptTemplate.from_template(prompt_template)

    # Template for refining the summary with additional context
    refine_template = (
        "Your job is to produce a final summary\n"
        # "We have provided an existing summary up to a certain point: {existing_answer}\n"
        "We have the opportunity to refine the existing summary"
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{text}\n"
        "------------\n"
        "Given the new context, refine the original summary in English"
        "If the context isn't useful, return the original summary."
    )
    refine_prompt = PromptTemplate.from_template(refine_template)

    # Loading the summarization chain with HuggingFaceHub model
    chain2 = load_summarize_chain(
        llm=HuggingFaceHub(
            repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
            model_kwargs={"temperature": 1, "max_length": 10000},
            huggingfacehub_api_token=api_token  # Assuming 'api_token' is defined somewhere
        ),
        chain_type="refine",
        question_prompt=prompt,
        refine_prompt=refine_prompt,
        return_intermediate_steps=False,
        input_key="input_documents",
        output_key="output_text",
    )

    return chain2

# Uncomment the line below if you want to execute the chain immediately
# result = chain2({"input_documents": split_docs}, return_only_outputs=True)


# Summarization Chain for ArXiv Papers (CHAIN 3)

## Function: `chain3()`

### Overview
The `chain3` function creates and returns a summarization chain tailored for processing arXiv papers. This chain is designed to generate concise summaries of the given text, with each summary limited to two sentences.

### Components
1. **Prompt Templates**
   - **Initial Summary Prompt**: A template instructing the generation of a summary based on the content of the arXiv paper.
   - **Refinement Prompt**: A template guiding the refinement of an existing summary by incorporating additional context.

2. **Summarization Chain Configuration**
   - **Language Model (LLM)**: Utilizes the Hugging Face model "mistralai/Mixtral-8x7B-Instruct-v0.1" with specified model parameters.
   - **Prompt Loading**: Initializes prompts for generating initial summaries and refining existing ones.
   - **Summarization Chain Creation**: Constructs a summarization chain using the configured components.
   - **Output Specification**: Defines the input and output keys for the chain.

### Usage
The function returns the configured summarization chain, allowing users to apply it to input documents for summarization. Example usage is provided in the commented-out section.

Note: Ensure that the required Hugging Face API token (`api_token`) is provided for authentication.


In [ ]:
########## CHAIN 3 arxiv_paper_1

# Define a function named chain3 that creates and returns a summarization chain for arXiv papers.
def chain3():
    
    # Define a template for the initial prompt that will be used to generate summaries from text snippets.
    prompt_template = """You are being given a markdown document with headers, this is part of a larger arxiv paper. Your job is to write a summary of the document such that every summary of the text is of 2 sentences
    here is the content of the section:
    "{text}"

    SUMMARY:"""
    
    # Create a PromptTemplate object from the prompt template.
    prompt = PromptTemplate.from_template(prompt_template)

    # Define a template for the refining prompt that will be used to synthesize snippets into a coherent paper summary.
    refine_template = ("""You are presented with a collection of text snippets. Each snippet is a summary of a specific section from an academic paper published on arXiv. Your objective is to synthesize these snippets into a coherent, concise summary of the entire paper.

        DOCUMENT SNIPPETS:
        "{text}"

        INSTRUCTIONS: Craft a concise summary below, capturing the essence of the paper based on the provided snippets.
        It is also important that you highlight the key contributions of the paper, and 3 key takeaways from the paper.
        Lastly, you should provide a list of 5 questions that you would ask the author of the paper if you had the chance. Remove all the backslash n (\n)
        SUMMARY:
        """
        )

    # Create a PromptTemplate object from the refining prompt template.
    refine_prompt = PromptTemplate.from_template(refine_template)
    
    # Load a summarization chain using HuggingFaceHub model with specified parameters.
    chain3 = load_summarize_chain(
        llm=HuggingFaceHub(repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
                        model_kwargs={"temperature":1, "max_length":10000},
                        huggingfacehub_api_token=api_token),
        chain_type="refine",
        question_prompt=prompt,
        refine_prompt=refine_prompt,
        return_intermediate_steps=False,
        input_key="input_documents",
        output_key="output_text",
    )
    
    # Return the summarization chain.
    return chain3

# Uncomment the following lines if you want to run the chain and print the result.
# result = chain3({"input_documents": split_docs}, return_only_outputs=True)
# print(result["output_text"])


# Summarization Chain Selection Function

## Function: `chain_function(checkbox_values)`

### Overview
The `chain_function` is designed to select and return the appropriate summarization chain based on the specified document types. It takes a list of checkbox values as input and returns the chosen summarization chain or an error message.

### Parameters
- `checkbox_values` (List[str]): List of selected document types.

### Returns
- Union[HuggingFaceHub, str]: Summarization chain corresponding to the selected document type or an error message.

### Document Types and Chains
- **Research Paper**: Utilizes `chain3()` for summarizing arXiv papers.
- **Legal Document**: Utilizes `chain2()` for summarizing legal documents.
- **Study Material**: Utilizes `chain1()` for summarizing study materials.

### Usage
Call the function with a list of selected document types to obtain the relevant summarization chain or an error message.

Example:
```python
selected_chain = chain_function(["Research Paper", "Legal Document"])


In [ ]:
# Define a function named `chain_function` that selects and returns the appropriate summarization chain based on the specified document types.
def chain_function(checkbox_values):
    
    # Overview comment for the function.
    """
    Select and return the appropriate summarization chain based on checkbox values.

    Parameters:
    - checkbox_values (List[str]): List of selected document types.

    Returns:
    - Union[HuggingFaceHub, str]: Summarization chain or an error message.
    """

    # Check if "Research Paper" is selected in the document types.
    if "Research Paper" in checkbox_values:
        # Call the `chain3` function to get the summarization chain for arXiv papers.
        output = chain3()
    
    # Check if "Legal Document" is selected in the document types.
    elif "Legal Document" in checkbox_values:
        # Call the `chain2` function to get the summarization chain for legal documents.
        output = chain2()
    
    # Check if "Study Material" is selected in the document types.
    elif "Study Material" in checkbox_values:
        # Call the `chain1` function to get the summarization chain for study materials.
        output = chain1()
    
    # If none of the valid document types are selected, provide an error message.
    else:
        output = "Please select a document type to run."
    
    # Return the selected summarization chain or error message.
    return output


# Summarization Result Generation Function

## Function: `result(chain, split_docs)`

### Overview
The `result` function utilizes a given summarization chain to generate summaries for a list of provided documents. It returns the concatenated summary text.

### Parameters
- `chain` (HuggingFaceHub): Summarization chain.
- `split_docs` (List): List of documents.

### Returns
- str: Concatenated summary text.

### Workflow
1. **Summarization Loop**: Iterates through each document in the list and applies the summarization chain.
2. **Summary Aggregation**: Concatenates individual summaries into a single text.

### Usage
Call the function with the summarization chain and a list of documents to obtain the concatenated summary text.



In [ ]:
# Define a function named `result` that generates summaries using the provided summarization chain.
def result(chain, split_docs):
  
   # Overview comment for the function.
    """
    Generate summaries using the provided summarization chain.

    Parameters:
    - chain (HuggingFaceHub): Summarization chain.
    - split_docs (List): List of documents.

    Returns:
    - str: Concatenated summary text.
    """
    
    # Initialize an empty list to store individual document summaries.
    summaries = []
    
    # Iterate through each document in the list of split documents.
    for doc in split_docs:
        # Use the summarization chain to generate a summary for each document.
        result = chain({"input_documents": [doc]})
        # Append the generated summary to the list of summaries.
        summaries.append(result["output_text"])
    
    # Initialize an empty string to concatenate all the individual summaries.
    text_concat = ""
    
    # Iterate through each summary and concatenate it to the overall summary text.
    for i in summaries:
      text_concat += i
    
    # Return the concatenated summary text.
    return text_concat


In [ ]:
title = """<p style="font-family:Century Gothic; text-align:center; font-size: 100px">S  I  M  P  L  I  F  Y</p>"""

# description = r"""<p style="font-family: Century Gothic; text-align:center; font-size: 100px">S  I  M  P  L  I  F  Y</p>
# """

# article = r"""
# If PhotoMaker is helpful, please help to ⭐ the <a href='https://github.com/TencentARC/PhotoMaker' target='_blank'>Github Repo</a>. Thanks!
# [![GitHub Stars](https://img.shields.io/github/stars/TencentARC/PhotoMaker?style=social)](https://github.com/TencentARC/PhotoMaker)
# ---
# 📝 **Citation**
# <br>
# If our work is useful for your research, please consider citing:
# ```bibtex
# @article{li2023photomaker,
#   title={PhotoMaker: Customizing Realistic Human Photos via Stacked ID Embedding},
#   author={Li, Zhen and Cao, Mingdeng and Wang, Xintao and Qi, Zhongang and Cheng, Ming-Ming and Shan, Ying},
#   booktitle={arXiv preprint arxiv:2312.04461},
#   year={2023}
# }
# ```
# 📋 **License**
# <br>
# Apache-2.0 LICENSE. Please refer to the [LICENSE file](https://huggingface.co/TencentARC/PhotoMaker/blob/main/LICENSE) for details.
# 📧 **Contact**
# <br>
# If you have any questions, please feel free to reach me out at <b>zhenli1031@gmail.com</b>.
# """

# tips = r"""
# ### Usage tips of PhotoMaker
# 1. Upload more photos of the person to be customized to **improve ID fidelty**. If the input is Asian face(s), maybe consider adding 'asian' before the class word, e.g., `asian woman img`
# 2. When stylizing, does the generated face look too realistic? Adjust the **Style strength** to 30-50, the larger the number, the less ID fidelty, but the stylization ability will be better.
# 3. If you want to generate realistic photos, you could try switching to our other gradio application [PhotoMaker](https://huggingface.co/spaces/TencentARC/PhotoMaker).
# 4. For **faster** speed, reduce the number of generated images and sampling steps. However, please note that reducing the sampling steps may compromise the ID fidelity.
# """


# File Processing Function

## Function: `process_file(list_file_obj)`

### Overview
The `process_file` function processes the uploaded file and returns the file path.

### Parameters
- `list_file_obj` (List): List of file objects.

### Returns
- str: Processed file path.

### Workflow
1. **Destination Path**: Specifies the destination path for saving the file (replace with your desired path).
2. **Copy to Destination**: Copies the uploaded file to the specified destination path.
3. **Return Path**: Returns the processed file path.

### Usage
Call the function with a list of file objects to obtain the processed file path.




In [ ]:
# def process_file(file_obj):
#     destination_path = "/content/sample_data"  # Replace with your desired path
#     shutil.copy(file_obj, destination_path)  # Save file to specified path
#     return os.path.join(destination_path, file_obj)
def process_file(list_file_obj):
    #Overview comment for the function.
    
      """
    Process the uploaded file and return the file path.

    Parameters:
    - list_file_obj (List): List of file objects.

    Returns:
    - str: Processed file path.
    """
    
    # list_file_path = [x.name for x in list_file_obj if x is not None]
    # file_content = file_obj.data
    # with tempfile.TemporaryFile() as temp_file:
    #     temp_file.write(file_content)
    #     temp_file_path = temp_file.name
    
    # Extract the name of the first file object in the list.
    # Assuming the list contains only one file object, use list_file_obj[0].

    return list_file_obj[0].name

# Inference Function

## Function: `inference(checkbox_values, uploaded_file)`

### Overview
The `inference` function performs inference based on selected document types and an uploaded file. It returns the summary text.

### Parameters
- `checkbox_values` (List[str]): List of selected document types.
- `uploaded_file` (List): List of uploaded file objects.

### Returns
- str: Summary text.

### Workflow
1. **File Processing**: Utilizes the `process_file` function to process the uploaded file.
2. **Data Ingestion**: Uses the processed file path to extract data and split into documents using the `data_ingestion` function.
3. **Summarization Chain Selection**: Chooses the appropriate summarization chain based on the selected document types using the `chain_function` function.
4. **Summarization**: Generates summaries for the split documents using the `result` function.
5. **Return Summary**: Returns the generated summary text.

### Usage
1. Launch the Gradio interface.
2. Choose the document type checkboxes and upload the file.
3. Click the "Submit" button to perform inference and display the summary text.



In [ ]:
# Define a function named `inference` that performs inference and returns the summary text.
def inference(checkbox_values, uploaded_file):
    
    # Overview comment for the function.
    """
    Perform inference and return the summary text.

    Parameters:
    - checkbox_values (List[str]): List of selected document types.
    - uploaded_file (List): List of uploaded file objects.

    Returns:
    - str: Summary text.
    """
    
    # Process the uploaded file and obtain the file path.
    file_path = process_file(uploaded_file)
    
    # Ingest the data from the processed file and split it into documents.
    split_docs = data_ingestion(file_path)
    
    # Select the appropriate summarization chain based on the selected document types.
    chain = chain_function(checkbox_values)
    
    # Generate the summary using the selected chain and the split documents.
    summary = result(chain, split_docs)
    
    # Return the generated summary text.
    return summary

# Create a Blocks demo for the user interface.
with gr.Blocks(theme="monochrome") as demo:
    
    # Markdown title for the demo.
    gr.Markdown(title)

    # Create a row for user input elements.
    with gr.Row():
        
        # Create a column for checkbox group and file upload elements.
        with gr.Column():
            # Checkbox group for selecting document types.
            checkbox_values = gr.CheckboxGroup(["Research Paper", "Legal Document", "Study Material"], label="Choose the document type")
            # File upload element for uploading files.
            uploaded_file = gr.Files(height=100, file_count="multiple", file_types=["text", ".docx", "pdf"], interactive=True, label="Upload your File.")
            # Button for submitting the form.
            btn = gr.Button("Submit")  # Place the button outside the Row for vertical alignment
        
        # Create a column for the text box element.
        with gr.Column():
            # Textbox for displaying the generated summary.
            txt = gr.Textbox(
                show_label=False,
                # placeholder="Simplify."
            )

    # Define the click event for the submit button.
    btn.click(
        fn=inference,
        inputs=[checkbox_values, uploaded_file],
        outputs=[txt],
        queue=False
    )

# Launch the demo with debugging enabled.
demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://73a4d3acc22d979d36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureW